In [1]:
import numpy as np
import pandas as pd

In [2]:
data = pd.read_csv("data_analytics.csv")[["Event Date", "Subscriber ID"]]
data["Event Date"] = pd.to_datetime(data["Event Date"])

In [8]:
def perform_calculations(
        data: str, price: float, fee: float, with_retention=True
        ):
    data = (data
            .groupby(by="Subscriber ID").count()
            .reset_index()
            .groupby(by="Event Date").count()
            .sort_index()
            )["Subscriber ID"]
    weeks, user_counts = data.index.values - 1, data.values

    if with_retention:
        users = user_counts[::-1].cumsum()[::-1]
        retention = users[1:] / users[0:-1]
        ltv = retention.cumprod().sum() * price * (1 - fee)
    else:
        ltv = (
            weeks * user_counts * price * (1 - fee)
        ).sum() / user_counts.sum()

    return ltv

In [9]:
%timeit perform_calculations(data, 9.99, 0.3, True)

1.26 ms ± 2.63 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [10]:
%timeit perform_calculations(data, 9.99, 0.3, False)

1.26 ms ± 3.83 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
